<a href="https://colab.research.google.com/github/EduardLouis/IMOB/blob/main/Olx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import csv
import requests
from bs4 imprt BeautifulSoup
import re
from datetime import datetime, date

In [ ]:
# Define a dictionary to map Romanian month names to English ones
romanian_to_english_months = {
    'ianuarie': 'January',
    'februarie': 'February',
    'martie': 'March',
    'aprilie': 'April',
    'mai': 'May',
    'iunie': 'June',
    'iulie': 'July',
    'august': 'August',
    'septembrie': 'September',
    'octombrie': 'October',
    'noiembrie': 'November',
    'decembrie': 'December'
}

In [ ]:
def get_url(nr_camere_filter, etaj_url_filter, supr_min, supr_max, an_const_url_filter):
    return f"https://www.olx.ro/imobiliare/apartamente-garsoniere-de-vanzare/{nr_camere_filter}/iasi-judet/?currency=EUR&search%5Border%5D=created_at:desc&search%5Bfilter_float_m:from%5D={supr_min}&search%5Bfilter_float_m:to%5D={supr_max}{an_const_url_filter}{etaj_url_filter}"


def get_number_of_pages(soup):
    pagination_wrapper = soup.find('div', {'data-testid': 'pagination-wrapper'})
    if not pagination_wrapper:
        return None
    pagination_items = pagination_wrapper.find_all('li', {'data-testid': 'pagination-list-item'})

    if pagination_items:  # Add this check
        return max([int(item.find('a')['data-testid'].split('-')[-1]) for item in pagination_items])
    else:
        return 1

In [ ]:
def extract_info_from_page(soup):
    listings = soup.find_all(class_='css-1sw7q4x')
    rows = []

    for listing in listings:
        title = listing.find('h6')
        if not title:
            continue

        link = str(listing.find('a')['href'])
        if link.startswith("/d/"):
            link = 'https://www.olx.ro'+link

        location_date = listing.find(class_='css-veheph').get_text()
        location = location_date.split(' - ')[0]
        price = listing.find(class_='css-10b0gli').get_text()
        price = float("".join(price.split(' ')[:2]))

        update_date = location_date.split(' - ')[1]

        # Extract and format the 'Update Date' using regular expressions

        match = re.search(r'\d{1,2} \w+ \d{4}', update_date)
        if match:
            date_str = match.group(0)
            # Replace Romanian month names with English ones
            for ro_month, en_month in romanian_to_english_months.items():
                date_str = date_str.replace(ro_month, en_month)
            parsed_date = datetime.strptime(date_str, '%d %B %Y')
            formatted_date = parsed_date.strftime('%d %B %Y')
        elif "Azi" in update_date:
            formatted_date = date.today()
        else:
            formatted_date = ''
        surface = float(listing.find(
            class_='css-643j0o').get_text().split(' ')[0])
        
        # Calculate the price per square meter
        price_per_sqm = price / surface

        data = {
            'Title': title.get_text(),
            'Link': link,
            'Location': location,
            'Price': price,
            'Update Date': formatted_date,
            'Surface': surface,
            'Price per sqm': price_per_sqm  # Add the new field here
        }
        rows.append(data)

    return rows

In [ ]:
def save_to_csv(rows, output_file_path):
    with open(output_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Title', 'Link', 'Location', 'Price', 'Update Date', 'Surface', 'Price per sqm']  # Add the new field here
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()
        for row in rows:
            writer.writerow(row)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# file
file_path = '/content/drive/MyDrive/Colab Notebooks/'

Mounted at /content/drive


In [ ]:
script_directory = os.path.dirname(file_path)
output_file_path = os.path.join(script_directory, 'output.csv')
anConstNameList = ["inainte-de-1977", "1977-1990", "1990-2000", "dupa-2000"]
an_const_url_list = ["&search%5Bfilter_enum_constructie%5D%5B{}%5D={}".format(
    xId, x) for xId, x in enumerate(anConstNameList)]


etajNameList = ["parter", "fl_1", "fl_2", "fl_3", "fl_4"]
etaj_url_list = ["&search%5Bfilter_enum_floor%5D%5B{}%5D={}".format(
    xId, x) for xId, x in enumerate(etajNameList)]

# Select the needed filters
nr_camere_filter = "2-camere"
an_const_url_filter = "".join(an_const_url_list)  # This line filters for "inainte-de-1977" and "1977-1990"
etaj_url_filter = "".join(etaj_url_list)  # This line filters for "parter" and "fl_1"
print(an_const_url_filter)
supr_min = 35
supr_max = 45
url = get_url(nr_camere_filter, etaj_url_filter, supr_min, supr_max, an_const_url_filter)
response = requests.get(url)
print(url)

&search%5Bfilter_enum_constructie%5D%5B0%5D=inainte-de-1977&search%5Bfilter_enum_constructie%5D%5B1%5D=1977-1990&search%5Bfilter_enum_constructie%5D%5B2%5D=1990-2000&search%5Bfilter_enum_constructie%5D%5B3%5D=dupa-2000
https://www.olx.ro/imobiliare/apartamente-garsoniere-de-vanzare/2-camere/iasi-judet/?currency=EUR&search%5Border%5D=created_at:desc&search%5Bfilter_float_m:from%5D=35&search%5Bfilter_float_m:to%5D=45&search%5Bfilter_enum_constructie%5D%5B0%5D=inainte-de-1977&search%5Bfilter_enum_constructie%5D%5B1%5D=1977-1990&search%5Bfilter_enum_constructie%5D%5B2%5D=1990-2000&search%5Bfilter_enum_constructie%5D%5B3%5D=dupa-2000&search%5Bfilter_enum_floor%5D%5B0%5D=parter&search%5Bfilter_enum_floor%5D%5B1%5D=fl_1&search%5Bfilter_enum_floor%5D%5B2%5D=fl_2&search%5Bfilter_enum_floor%5D%5B3%5D=fl_3&search%5Bfilter_enum_floor%5D%5B4%5D=fl_4


In [ ]:
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    number_of_pages = get_number_of_pages(soup)
    if not number_of_pages: 
        print(f"Nici un rezult.")
    else: 

        print(f"Number of pages: {number_of_pages}")

        rows = []
        for pg_id in range(1, number_of_pages + 1):
            print(f"Extract info from page: {pg_id}")
            url = get_url(nr_camere_filter, etaj_url_filter, supr_min, supr_max, an_const_url_filter)
            url += f"&page={pg_id}"
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                rows.extend(extract_info_from_page(soup))
            else:
                print(f"Failed to retrieve page {pg_id}. Status code: {response.status_code}")

        print("Export info.")
        save_to_csv(rows, output_file_path)

else:
    print(f"Failed to retrieve the first page. Status code: {response.status_code}")

Number of pages: 11
Extract info from page: 1
Extract info from page: 2
Extract info from page: 3
Extract info from page: 4
Extract info from page: 5
Extract info from page: 6
Extract info from page: 7
Extract info from page: 8
Extract info from page: 9
Extract info from page: 10
Extract info from page: 11
Export info.


In [ ]:

import json
from datetime import date

def save_to_json(data, output_file_path):
    def convert_to_serializable(obj):
        if isinstance(obj, date):
            return obj.strftime('%Y-%m-%d')
        raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')

    with open(output_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, default=convert_to_serializable)

# Assuming you have the 'rows' list containing the extracted data
output_file_pathJSON = os.path.join(script_directory, 'output.json')
save_to_json(rows, output_file_pathJSON)

In [ ]:
!pip install -q streamlit